In [42]:
import pandas as pd
from pulp import *

In [43]:
inputs = pd.read_csv('inputs.csv', index_col=0, delimiter=';')
outputs = pd.read_csv('outputs.csv', index_col=0, delimiter=';')

In [44]:
efficiency_scores = {}
reference_units = {}
adjustments = {}

In [45]:
for DMUo in inputs.index:
    problem = LpProblem(f"Efficiency_{DMUo}", LpMinimize)

    lambdas = LpVariable.dicts("Lambda", (i for i in inputs.index), lowBound=0)
    theta = LpVariable("theta", lowBound=0, upBound=1)

    problem += theta

    for input_metric in inputs.columns:
        problem += lpSum([lambdas[i] * inputs.loc[i, input_metric] for i in inputs.index]) <= theta * inputs.loc[DMUo, input_metric]

    for output_metric in outputs.columns:
        problem += lpSum([lambdas[i] * outputs.loc[i, output_metric] for i in outputs.index]) >= outputs.loc[DMUo, output_metric]

    problem.solve(PULP_CBC_CMD(msg=False))

    if LpStatus[problem.status] == 'Optimal':
        efficiency_score = value(theta)
        efficiency_scores[DMUo] = efficiency_score

        if efficiency_score < 1:
            reference_units[DMUo] = {}
            adjustments[DMUo] = {}
            for i in inputs.index:
                if lambdas[i].varValue > 0:
                    reference_units[DMUo][i] = lambdas[i].varValue

            for input_metric in inputs.columns:
                optimal_input = sum(lambdas[i].varValue * inputs.loc[i, input_metric] for i in inputs.index)
                adjustments[DMUo][input_metric] = optimal_input

    else:
        print(f"Problem for {DMUo} is not solvable")

for airport, efficiency in efficiency_scores.items():
    print(f"Efficiency for {airport}: {efficiency:.2f}")
    if airport in adjustments:
        print(f"Adjustments for {airport}:")
        for metric, value in adjustments[airport].items():
            print(f"  {metric}: reduce to {value:.2f}")


Efficiency for WAW: 1.00
Efficiency for KRK: 1.00
Efficiency for KAT: 0.59
Adjustments for KAT:
  i1: reduce to 2.13
  i2: reduce to 18.92
  i3: reduce to 33.94
  i4: reduce to 4.40
Efficiency for WRO: 1.00
Efficiency for POZ: 0.80
Adjustments for POZ:
  i1: reduce to 1.20
  i2: reduce to 8.00
  i3: reduce to 19.20
  i4: reduce to 1.93
Efficiency for LCJ: 0.30
Adjustments for LCJ:
  i1: reduce to 0.18
  i2: reduce to 2.78
  i3: reduce to 7.20
  i4: reduce to 0.47
Efficiency for GDN: 1.00
Efficiency for SZZ: 0.27
Adjustments for SZZ:
  i1: reduce to 0.19
  i2: reduce to 2.71
  i3: reduce to 6.96
  i4: reduce to 0.47
Efficiency for BZG: 1.00
Efficiency for RZE: 0.41
Adjustments for RZE:
  i1: reduce to 0.25
  i2: reduce to 2.46
  i3: reduce to 4.62
  i4: reduce to 0.54
Efficiency for IEG: 0.26
Adjustments for IEG:
  i1: reduce to 0.03
  i2: reduce to 0.39
  i3: reduce to 1.11
  i4: reduce to 0.06
